In [ ]:
#-------------------------------------------------------------------------------------JUPYTER NOTEBOOK SETTINGS-------------------------------------------------------------------------------------
from IPython.core.display import display, HTML                                    
display(HTML("<style>.container { width:100% !important; }</style>"))  
import IPython.display as display

In [ ]:
import os
import gc
import re
import librosa
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from joblib import dump, load

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Layer, Input, Conv1D, MaxPooling1D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping 
from tensorflow.keras import mixed_precision

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.config.list_physical_devices('GPU')

In [ ]:
# Set up mixed precision policy
mixed_precision.set_global_policy('mixed_float16')

### Data Loading and Processing

In [ ]:
# Load the saved data
mfcc_features = load('saved_data/mfcc_features.joblib')
labels = load('saved_data/labels.joblib')
gender_labels = load('saved_data/gender_labels.joblib')

# Define SpecAugment class
class SpecAugment:
    def __init__(self, spectrogram_shape, mF_num_freq_masks, F_freq_mask_max_consecutive, mT_num_time_masks, T_time_mask_max_consecutive, enable_time_warp, W_time_warp_max_distance, mask_with_mean):
        self.spectrogram_shape = spectrogram_shape
        self.mF_num_freq_masks = mF_num_freq_masks
        self.F_freq_mask_max_consecutive = F_freq_mask_max_consecutive
        self.mT_num_time_masks = mT_num_time_masks
        self.T_time_mask_max_consecutive = T_time_mask_max_consecutive
        self.enable_time_warp = enable_time_warp
        self.W_time_warp_max_distance = W_time_warp_max_distance
        self.mask_with_mean = mask_with_mean

    def mapper(self):
        def augment_spec(spec, label, gender_label):
            # Apply frequency masks
            for _ in range(self.mF_num_freq_masks):
                spec = self.apply_freq_mask(spec)
            # Apply time masks
            for _ in range(self.mT_num_time_masks):
                spec = self.apply_time_mask(spec)
            return spec, label, gender_label

        return lambda spec, label, gender_label: tf.numpy_function(augment_spec, [spec, label, gender_label], [tf.float32, tf.string, tf.string])

    def apply_freq_mask(self, spec):
        freq_mask = np.random.randint(0, self.spectrogram_shape[0], self.mF_num_freq_masks)
        for mask in freq_mask:
            f_start = mask
            f_end = min(f_start + self.F_freq_mask_max_consecutive, self.spectrogram_shape[0])
            if self.mask_with_mean:
                spec[f_start:f_end, :] = np.mean(spec)
            else:
                spec[f_start:f_end, :] = 0
        return spec

    def apply_time_mask(self, spec):
        time_mask = np.random.randint(0, self.spectrogram_shape[1], self.mT_num_time_masks)
        for mask in time_mask:
            t_start = mask
            t_end = min(t_start + self.T_time_mask_max_consecutive, self.spectrogram_shape[1])
            if self.mask_with_mean:
                spec[:, t_start:t_end] = np.mean(spec)
            else:
                spec[:, t_start:t_end] = 0
        return spec

# Apply SpecAugment
sa = SpecAugment(spectrogram_shape=[mfcc_features.shape[1], mfcc_features.shape[2]], mF_num_freq_masks=3, F_freq_mask_max_consecutive=4, mT_num_time_masks=1, T_time_mask_max_consecutive=1, enable_time_warp=False, W_time_warp_max_distance=6, mask_with_mean=True)

# Split the data into train/test/val with 70/15/15 split
train_features, temp_features, train_labels, temp_labels, train_gender_labels, temp_gender_labels = train_test_split(
    mfcc_features, labels, gender_labels, test_size=0.3, random_state=42, stratify=labels)

val_features, test_features, val_labels, test_labels, val_gender_labels, test_gender_labels = train_test_split(
    temp_features, temp_labels, temp_gender_labels, test_size=0.5, random_state=42, stratify=temp_labels)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_features, train_labels, train_gender_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((val_features, val_labels, val_gender_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_features, test_labels, test_gender_labels))

# Apply SpecAugment to the datasets
train_dataset = train_dataset.map(sa.mapper(), num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(sa.mapper(), num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(sa.mapper(), num_parallel_calls=tf.data.AUTOTUNE)

# Batch and prefetch the datasets
batch_size = 16
train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Print dataset sizes for verification
print(f"Train dataset size (features): {len(train_features)}")
print(f"Train dataset size (labels): {len(train_labels)}")
print(f"Train dataset size (gender labels): {len(train_gender_labels)}")

print(f"Validation dataset size (features): {len(val_features)}")
print(f"Validation dataset size (labels): {len(val_labels)}")
print(f"Validation dataset size (gender labels): {len(val_gender_labels)}")

print(f"Test dataset size (features): {len(test_features)}")
print(f"Test dataset size (labels): {len(test_labels)}")
print(f"Test dataset size (gender labels): {len(test_gender_labels)}")

In [ ]:
# Flatten the MFCC features for the first 10 samples for display purposes
x_flattened_for_display = [x[i].flatten()[:25] for i in range(25)]  # Display only the first 10 MFCC coefficients of each sample

# Create a DataFrame with the flattened MFCC features and labels
df = pd.DataFrame(x_flattened_for_display)
df['Label'] = y[:25]  # Add the labels as the last column

# Setting column names for clarity in display
feature_columns = [f'Feature_{i+1}' for i in range(df.shape[1] - 1)]  # Feature column names
df.columns = feature_columns + ['Label']  # Rename the columns for better understanding

# Display the DataFrame in Jupyter Notebook
df

### CNN Setup and Training

In [ ]:
# ONEHOT ENCODING THE LABELS
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

# Convert labels to one-hot encoding
y_train_onehot = to_categorical(y_train_encoded)
y_val_onehot = to_categorical(y_val_encoded)
y_test_onehot = to_categorical(y_test_encoded)

In [ ]:
# Simplified Primary model
input_shape = (13, 398)
input_layer = Input(shape=input_shape)
x = Conv1D(32, kernel_size=3, activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = MaxPooling1D(pool_size=2)(x)
x = Dropout(0.25)(x)
x = Flatten()(x)

# Task-specific classifier
task_output = Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
task_output = Dropout(0.5)(task_output)
task_output = Dense(y_train_onehot.shape[1], activation='softmax', name='task_output')(task_output)

# Create the model
model = Model(inputs=input_layer, outputs=task_output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
early_stopping_monitor = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True, verbose=1)
reduce_lr_on_plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=20, min_lr=0.0000001, verbose=1)

# Train the model
history = model.fit(
    x_train,
    y_train_onehot,
    epochs=50,
    batch_size=1024,
    validation_data=(x_val, y_val_onehot),
    callbacks=[early_stopping_monitor, reduce_lr_on_plateau],
    verbose=1  
)

# Check the training history
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.show()

In [ ]:
# CONVOLUTIONAL NEURAL NETWORK SETUP AND TRAINING
def load_latest_weights(weights_dir, file_pattern):
    """Load the latest weights based on the file modification time."""
    # List all files in the directory that match the pattern
    all_weights = [os.path.join(weights_dir, f) for f in os.listdir(weights_dir) if file_pattern in f]
    # Find the most recent file by sorting based on modification time
    latest_weights = max(all_weights, key=os.path.getmtime, default=None)
    if latest_weights:
        print(f"Loading weights from {latest_weights}")
        return latest_weights
    else:
        print("No weights file found.")
        return None

# Gradient Reversal Layer
class GradientReversalLayer(Layer):
    def __init__(self, lambda_):
        super(GradientReversalLayer, self).__init__()
        self.lambda_ = lambda_

    @tf.custom_gradient
    def call(self, x):
        def grad(dy):
            return -self.lambda_ * dy
        return x, grad

# Domain Classifier
def create_domain_classifier(feature_extractor_output):
    x = GradientReversalLayer(lambda_=1.0)(feature_extractor_output)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    domain_output = Dense(1, activation='sigmoid', name='domain_output')(x)
    return domain_output

# Primary model
input_shape = (13, 398)
input_layer = Input(shape=input_shape)
x = Conv1D(32, kernel_size=3, activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv1D(32, kernel_size=3, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling1D(pool_size=2)(x)
x = Dropout(0.25)(x)
x = Conv1D(64, kernel_size=2, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv1D(64, kernel_size=2, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling1D(pool_size=2)(x)
x = Dropout(0.25)(x)
feature_extractor_output = Flatten()(x)

# Task-specific classifier
task_output = Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(feature_extractor_output)
task_output = Dropout(0.5)(task_output)
task_output = Dense(y_train_onehot.shape[1], activation='softmax', name='task_output')(task_output)

# Domain classifier
domain_output = create_domain_classifier(feature_extractor_output)

# Create the combined model
model = Model(inputs=input_layer, outputs=[task_output, domain_output])

# Compile the model with two losses
model.compile(optimizer=Adam(learning_rate=0.001),
              loss={'task_output': 'categorical_crossentropy', 'domain_output': 'binary_crossentropy'},
              metrics={'task_output': 'accuracy', 'domain_output': 'accuracy'})

# Callbacks
early_stopping_monitor = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True, verbose=1)
reduce_lr_on_plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=20, min_lr=0.0000001, verbose=1)

class SaveWeightsCallback(Callback):
    def __init__(self, save_freq, filepath):
        super(SaveWeightsCallback, self).__init__()
        self.save_freq = save_freq
        self.filepath = filepath
    
    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_freq == 0:
            self.model.save_weights(self.filepath.format(epoch=epoch + 1))

weights_saver = SaveWeightsCallback(save_freq=50, filepath='saved_data/models/prototype_4/adversarial-training_custom-cnn_medium-masked-data/adversarial-training_custom-cnn_weights_epoch-{epoch}.weights.h5')

# Train the model
num_epochs_per_stage = 50
total_epochs = 500
current_epoch = 0
all_history = []

while current_epoch < total_epochs:
    try:
        latest_weights_file = load_latest_weights('saved_data', '.weights.h5')
        if latest_weights_file:
            model.load_weights(latest_weights_file)
        
        # Select only samples with valid gender labels for domain training
        valid_idx = genders_train != -1
        x_train_valid = x_train[valid_idx]
        y_train_onehot_valid = y_train_onehot[valid_idx]
        genders_train_valid = genders_train[valid_idx]

        history = model.fit(
            x_train,
            {'task_output': y_train_onehot, 'domain_output': genders_train},
            epochs=current_epoch + num_epochs_per_stage,
            batch_size=1024,
            validation_data=(x_val, {'task_output': y_val_onehot, 'domain_output': genders_val}),
            callbacks=[weights_saver, early_stopping_monitor, reduce_lr_on_plateau],
            initial_epoch=current_epoch,
            verbose=1  
        )
        
        all_history.append(history.history)
        current_epoch += len(history.history['loss'])
        gc.collect()

        if early_stopping_monitor.stopped_epoch > 0:
            print(f"Early stopping triggered at epoch {current_epoch}")
            break

    except Exception as e:
        print("An error occurred during training:", e)
        break

In [ ]:
model.save('saved_data/models/prototype_4/adversarial-training_custom-cnn_medium-masked-data/adversarial-training_custom-cnn_final_model.keras')

if all_history:
    final_history = {key: np.concatenate([seg[key] for seg in all_history]) for key in all_history[0]}
    dump(final_history, 'saved_data/models/prototype_4/adversarial-training_custom-cnn_medium-masked-data/adversarial-training_custom-cnn_training_history.joblib')
else:
    print("No training history was recorded.")